# Sentiment Analysis: Donald Trump and Joe Biden Reelection Run in 2024

## Description

This part of the project will be the develop of the code to gather the real time data from the Twitter API. The data will be used to perform a sentiment analysis of the tweets about Donald Trump and Joe Biden reelection run in 2024.



## Gathering Data

In [5]:
# Importing libraries

import tweepy
from tweepy.streaming import 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import time
import os
import datetime
import json


ImportError: cannot import name 'StreamListener' from 'tweepy.streaming' (C:\Users\diazi\anaconda3\envs\scrap\lib\site-packages\tweepy\streaming.py)

In [3]:
## Write a class to gather the data from the Twitter API
class TwitterStreamListener(tweepy.StreamListener):
    def __init__(self, consumer_key, consumer_secret, access_token, access_token_secret, keywords):
        super().__init__()
        self.consumer_key = consumer_key
        self.consumer_secret = consumer_secret
        self.access_token = access_token
        self.access_token_secret = access_token_secret
        self.keywords = keywords
        self.file_name = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".csv"
        self.df = pd.DataFrame(columns=["id", "text", "created_at", "user_location", "place"])

    def on_status(self, status):
        if status.place is not None and "United States" in status.place.full_name:
            self.df = self.df.append({
                "id": status.id,
                "text": status.text,
                "created_at": status.created_at,
                "user_location": status.user.location,
                "place": status.place.full_name
            }, ignore_index=True)

    def on_error(self, status_code):
        if status_code == 420:
            return False

    def save_tweets(self):
        if not os.path.exists("data"):
            os.makedirs("data")
        self.df.to_csv("data/" + self.file_name, index=False)

    def start_stream(self):
        auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
        auth.set_access_token(self.access_token, self.access_token_secret)
        stream = tweepy.Stream(auth, self)
        stream.filter(track=self.keywords, is_async=True)

        while True:
            self.save_tweets()
            self.df = pd.DataFrame(columns=["id", "text", "created_at", "user_location", "place"])
            time.sleep(3600)  # wait 1 hour before saving more data

AttributeError: module 'tweepy' has no attribute 'StreamListener'

In [3]:
## load json file with keys
with open("C:/Users/diazi/.secret/twitter_api.json") as json_file:
    keys = json.load(json_file)
keys.keys()

dict_keys(['consumer-key', 'consumer-key-secret', 'access-token', 'access-token-secret', 'bearer-token'])

In [4]:
## set keys

consumer_key = keys["consumer-key"]
consumer_secret = keys["consumer-key-secret"]
access_token = keys["access-token"]
access_token_secret = keys["access-token-secret"]
keywords = ["election 2024", "presidential race 2024", "candidate 2024"]
location = "-126.33,24.84,-65.94,49.38"  # bounding box of the US


In [5]:
%%time
twitter_data = TwitterData(consumer_key, consumer_secret, access_token, access_token_secret, keywords, location)

while True:
    twitter_data.get_tweets()
    time.sleep(36)  # wait 1 hour before getting more data


KeyboardInterrupt: 